# Experiment 004: Per-Target Heterogeneous Model with Combined Features

Based on Loop 3 analysis:
- Per-target heterogeneous + combined features achieves **0.0662 MAE** on single solvent
- This is a **12% improvement** over current 0.0748
- SM → HistGradientBoostingRegressor (max_depth=7, max_iter=700, lr=0.04)
- Products → ExtraTreesRegressor (n_estimators=500, min_samples_leaf=2, max_depth=10)
- Feature weighting: 0.8 acs_pca + 0.2 spange

**TEMPLATE COMPLIANCE**: Last 3 cells are EXACTLY as template, NO cells after them.

In [1]:
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import HistGradientBoostingRegressor, ExtraTreesRegressor
from abc import ABC
import tqdm
import warnings
warnings.filterwarnings('ignore')

DATA_PATH = '/home/data'
torch.set_default_dtype(torch.double)
print("Setup complete")

Setup complete


In [2]:
# --- UTILITY FUNCTIONS ---
INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
TARGET_LABELS = ["Product 2", "Product 3", "SM"]

def load_data(name="full"):
    assert name in ["full", "single_solvent"]
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[["Residence Time", "Temperature", "SOLVENT NAME"]]
    Y = df[TARGET_LABELS]
    return X, Y

def load_features(name="spange_descriptors"):
    return pd.read_csv(f'{DATA_PATH}/{name}_lookup.csv', index_col=0)

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

# Load both feature sets
SPANGE_DF = load_features('spange_descriptors')
ACS_PCA_DF = load_features('acs_pca_descriptors')
print(f"Spange: {SPANGE_DF.shape}, ACS_PCA: {ACS_PCA_DF.shape}")
print(f"Spange solvents: {len(SPANGE_DF)}")
print(f"ACS_PCA solvents: {len(ACS_PCA_DF)}")

Spange: (26, 13), ACS_PCA: (24, 5)
Spange solvents: 26
ACS_PCA solvents: 24


In [3]:
# --- BASE CLASSES ---
class SmilesFeaturizer(ABC):
    def __init__(self): raise NotImplementedError
    def featurize(self, X): raise NotImplementedError

class BaseModel(ABC):
    def __init__(self): pass
    def train_model(self, X_train, y_train): raise NotImplementedError
    def predict(self): raise NotImplementedError

In [4]:
# --- DUAL FEATURIZER (Spange + ACS_PCA) ---
class DualFeaturizer:
    """Featurizer that builds features from both Spange and ACS_PCA descriptors."""
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange = SPANGE_DF
        self.acs_pca = ACS_PCA_DF

    def _build_process_features(self, X):
        """Build process features with Arrhenius kinetics."""
        rt = X['Residence Time'].values.astype(np.float64).reshape(-1, 1)
        temp = X['Temperature'].values.astype(np.float64).reshape(-1, 1)
        
        # Arrhenius kinetic features
        temp_k = temp + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(rt + 1e-6)
        interaction = inv_temp * log_time
        
        return np.hstack([rt, temp, inv_temp, log_time, interaction])

    def _get_solvent_features(self, X, feature_df, flip=False):
        """Get solvent features from a feature dataframe."""
        if self.mixed:
            A = feature_df.loc[X["SOLVENT A NAME"]].values
            B = feature_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                return B * (1 - pct) + A * pct
            else:
                return A * (1 - pct) + B * pct
        else:
            return feature_df.loc[X["SOLVENT NAME"]].values

    def featurize_spange(self, X, flip=False):
        """Build features using Spange descriptors."""
        process = self._build_process_features(X)
        solvent = self._get_solvent_features(X, self.spange, flip)
        if self.mixed:
            pct = X["SolventB%"].values.reshape(-1, 1)
            return np.hstack([process, pct, solvent])
        return np.hstack([process, solvent])

    def featurize_acs(self, X, flip=False):
        """Build features using ACS_PCA descriptors."""
        process = self._build_process_features(X)
        solvent = self._get_solvent_features(X, self.acs_pca, flip)
        if self.mixed:
            pct = X["SolventB%"].values.reshape(-1, 1)
            return np.hstack([process, pct, solvent])
        return np.hstack([process, solvent])

In [5]:
# --- PER-TARGET HETEROGENEOUS MODEL ---
class PerTargetHeterogeneousModel(BaseModel):
    """Per-target heterogeneous model with combined features.
    
    - SM: HistGradientBoostingRegressor (max_depth=7, max_iter=700, lr=0.04)
    - Products: ExtraTreesRegressor (n_estimators=500, min_samples_leaf=2, max_depth=10)
    - Feature weighting: 0.8 acs_pca + 0.2 spange
    """
    def __init__(self, data='single'):
        self.data_type = data
        self.featurizer = DualFeaturizer(mixed=(data=='full'))
        self.targets = ['Product 2', 'Product 3', 'SM']
        
        # Scalers for each feature set
        self.scaler_spange = StandardScaler()
        self.scaler_acs = StandardScaler()
        
        # Models: {target: {feature_set: model}}
        self.models = {}
        
        # Feature weights: 0.8 acs_pca + 0.2 spange
        self.acs_weight = 0.8
        self.spange_weight = 0.2

    def train_model(self, X_train, y_train):
        # Build features
        X_spange = self.featurizer.featurize_spange(X_train)
        X_acs = self.featurizer.featurize_acs(X_train)
        y = y_train.values
        
        if self.data_type == 'full':
            # Data augmentation with flipped features
            X_spange_flip = self.featurizer.featurize_spange(X_train, flip=True)
            X_acs_flip = self.featurizer.featurize_acs(X_train, flip=True)
            X_spange = np.vstack([X_spange, X_spange_flip])
            X_acs = np.vstack([X_acs, X_acs_flip])
            y = np.vstack([y, y])
        
        # Scale features
        X_spange_scaled = self.scaler_spange.fit_transform(X_spange)
        X_acs_scaled = self.scaler_acs.fit_transform(X_acs)
        
        # Train per-target models
        for i, target in enumerate(self.targets):
            y_target = y[:, i]
            
            if target == 'SM':
                # HistGradientBoosting for SM
                model_spange = HistGradientBoostingRegressor(
                    max_depth=7, max_iter=700, learning_rate=0.04, random_state=42
                )
                model_acs = HistGradientBoostingRegressor(
                    max_depth=7, max_iter=700, learning_rate=0.04, random_state=42
                )
            else:
                # ExtraTrees for Products
                model_spange = ExtraTreesRegressor(
                    n_estimators=500, min_samples_leaf=2, max_depth=10,
                    n_jobs=-1, random_state=42
                )
                model_acs = ExtraTreesRegressor(
                    n_estimators=500, min_samples_leaf=2, max_depth=10,
                    n_jobs=-1, random_state=42
                )
            
            model_spange.fit(X_spange_scaled, y_target)
            model_acs.fit(X_acs_scaled, y_target)
            
            self.models[target] = {
                'spange': model_spange,
                'acs': model_acs
            }

    def predict(self, X):
        if self.data_type == 'full':
            # TTA for mixed solvents
            X_spange = self.featurizer.featurize_spange(X, flip=False)
            X_spange_flip = self.featurizer.featurize_spange(X, flip=True)
            X_acs = self.featurizer.featurize_acs(X, flip=False)
            X_acs_flip = self.featurizer.featurize_acs(X, flip=True)
            
            X_spange_sc = self.scaler_spange.transform(X_spange)
            X_spange_flip_sc = self.scaler_spange.transform(X_spange_flip)
            X_acs_sc = self.scaler_acs.transform(X_acs)
            X_acs_flip_sc = self.scaler_acs.transform(X_acs_flip)
            
            preds = []
            for target in self.targets:
                # Spange predictions with TTA
                p_spange = (self.models[target]['spange'].predict(X_spange_sc) + 
                           self.models[target]['spange'].predict(X_spange_flip_sc)) / 2
                # ACS predictions with TTA
                p_acs = (self.models[target]['acs'].predict(X_acs_sc) + 
                        self.models[target]['acs'].predict(X_acs_flip_sc)) / 2
                # Weighted combination
                p_combined = self.acs_weight * p_acs + self.spange_weight * p_spange
                preds.append(p_combined.reshape(-1, 1))
        else:
            X_spange = self.featurizer.featurize_spange(X)
            X_acs = self.featurizer.featurize_acs(X)
            
            X_spange_sc = self.scaler_spange.transform(X_spange)
            X_acs_sc = self.scaler_acs.transform(X_acs)
            
            preds = []
            for target in self.targets:
                p_spange = self.models[target]['spange'].predict(X_spange_sc)
                p_acs = self.models[target]['acs'].predict(X_acs_sc)
                p_combined = self.acs_weight * p_acs + self.spange_weight * p_spange
                preds.append(p_combined.reshape(-1, 1))
        
        final_preds = np.hstack(preds)
        final_preds = np.clip(final_preds, 0, 1)
        return torch.tensor(final_preds, dtype=torch.double)

In [6]:
# --- QUICK VALIDATION TEST ---
print("Testing PerTargetHeterogeneousModel...")
X_test, Y_test = load_data("single_solvent")

# Quick leave-one-out test on first 5 solvents
errors = []
split_gen = generate_leave_one_out_splits(X_test, Y_test)
for i, ((train_X, train_Y), (test_X, test_Y)) in enumerate(split_gen):
    if i >= 5: break
    model = PerTargetHeterogeneousModel(data='single')
    model.train_model(train_X, train_Y)
    preds = model.predict(test_X).numpy()
    mae = np.mean(np.abs(preds - test_Y.values))
    errors.append(mae)
    solvent = test_X['SOLVENT NAME'].iloc[0]
    print(f"Fold {i} ({solvent}): MAE = {mae:.4f}")

print(f"\nQuick test MAE: {np.mean(errors):.4f}")

Testing PerTargetHeterogeneousModel...


Fold 0 (1,1,1,3,3,3-Hexafluoropropan-2-ol): MAE = 0.1446


Fold 1 (2,2,2-Trifluoroethanol): MAE = 0.0962


Fold 2 (2-Methyltetrahydrofuran [2-MeTHF]): MAE = 0.0369


Fold 3 (Acetonitrile): MAE = 0.0813


Fold 4 (Acetonitrile.Acetic Acid): MAE = 0.1077

Quick test MAE: 0.0934


## Template-Compliant Cross-Validation

The following 3 cells are the FINAL 3 cells - EXACTLY as in the template.

In [7]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

import tqdm

X, Y = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = PerTargetHeterogeneousModel(data='single') # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_single_solvent = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

0it [00:00, ?it/s]

1it [00:01,  1.87s/it]

2it [00:03,  1.92s/it]

3it [00:05,  1.90s/it]

4it [00:07,  1.93s/it]

...

21it [00:40,  1.95s/it]

22it [00:42,  1.97s/it]

23it [00:44,  1.94s/it]

24it [00:46,  1.94s/it]

24it [00:46,  1.95s/it]

In [8]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

X, Y = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = PerTargetHeterogeneousModel(data = 'full') # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_full_data = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

0it [00:00, ?it/s]

1it [00:02,  2.75s/it]

2it [00:05,  2.73s/it]

3it [00:08,  2.65s/it]

4it [00:10,  2.67s/it]

...

10it [00:25,  2.53s/it]

11it [00:28,  2.54s/it]

12it [00:30,  2.52s/it]

13it [00:33,  2.53s/it]

13it [00:33,  2.57s/it]

In [9]:
########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("submission.csv", index=True)

########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################